# Section 5.2: Using Torch with thermometer data

In [1]:
import numpy as np
import torch

In [2]:
t_c = [0.5,  14.0, 15.0, 28.0, 11.0,  8.0,  3.0, -4.0,  6.0, 13.0, 21.0]
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]
t_c = torch.tensor(t_c)
t_u = torch.tensor(t_u)

In [3]:
def model(t_u, w, b):
    return w*t_u + b

def loss_fn(t_p, t_c):
    return ((t_p - t_c)**2).mean()

In [4]:
params = torch.tensor([1.0, 0.0], requires_grad = True)
print(params.grad)

None


In [5]:
loss = loss_fn(model(t_u, *params), t_c)
loss.backward()
print(params.grad)

tensor([4517.2969,   82.6000])


In [6]:
def training_loop(epochs, learning_rate, params, t_u, t_c):
    for epoch in range(1, epochs + 1):
        if params.grad is not None: params.grad.zero_()
        t_p = model(t_u, *params)
        loss = loss_fn(t_p, t_c)
        loss.backward()
        with torch.no_grad():
            params -= learning_rate*params.grad
        if epoch % 500 == 0:
            print("Epoch: {}, Loss: {:.2f}".format(epoch, loss))
    return params

In [7]:
t_un = 0.1 * t_u
params = training_loop(5000, 1e-2, torch.tensor([1.0, 0.0], requires_grad = True), t_un, t_c)
print(params)

Epoch: 500, Loss: 7.86
Epoch: 1000, Loss: 3.83
Epoch: 1500, Loss: 3.09
Epoch: 2000, Loss: 2.96
Epoch: 2500, Loss: 2.93
Epoch: 3000, Loss: 2.93
Epoch: 3500, Loss: 2.93
Epoch: 4000, Loss: 2.93
Epoch: 4500, Loss: 2.93
Epoch: 5000, Loss: 2.93
tensor([  5.3671, -17.3012], requires_grad=True)


In [8]:
import torch.optim as optim
dir(optim)

['ASGD',
 'Adadelta',
 'Adagrad',
 'Adam',
 'AdamW',
 'Adamax',
 'LBFGS',
 'Optimizer',
 'RMSprop',
 'Rprop',
 'SGD',
 'SparseAdam',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'lr_scheduler']

In [9]:
params = torch.tensor([1.0, 0.0], requires_grad = True)
learning_rate = 1e-5
optimizer = optim.SGD([params], lr = learning_rate)
t_p = model(t_un, *params)
loss = loss_fn(t_p, t_c)
optimizer.zero_grad()
loss.backward()
optimizer.step()
print(params)

tensor([1.0008e+00, 1.0640e-04], requires_grad=True)


In [10]:
def training_loop(epochs, optimizer, params, t_u, t_c):
    for epoch in range(1, epochs + 1):
        t_p = model(t_u, *params)
        loss = loss_fn(t_p, t_c)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if epoch % 500 == 0:
            print("Epoch: {}, Loss: {:.2f}".format(epoch, loss))
    return params

params = torch.tensor([1.0, 0.0], requires_grad = True)
params = training_loop(5000, 
                       optim.SGD([params], lr = 1e-2), 
                       params, 
                       t_un, 
                       t_c)
print(params)

Epoch: 500, Loss: 7.86
Epoch: 1000, Loss: 3.83
Epoch: 1500, Loss: 3.09
Epoch: 2000, Loss: 2.96
Epoch: 2500, Loss: 2.93
Epoch: 3000, Loss: 2.93
Epoch: 3500, Loss: 2.93
Epoch: 4000, Loss: 2.93
Epoch: 4500, Loss: 2.93
Epoch: 5000, Loss: 2.93
tensor([  5.3671, -17.3012], requires_grad=True)


In [11]:
params = torch.tensor([1.0, 0.0], requires_grad = True)
params = training_loop(2000, 
                       optim.Adam([params], lr = 1e-1), 
                       params, 
                       t_un, 
                       t_c)
print(params)

Epoch: 500, Loss: 2.96
Epoch: 1000, Loss: 2.93
Epoch: 1500, Loss: 2.93
Epoch: 2000, Loss: 2.93
tensor([  5.3677, -17.3048], requires_grad=True)


In [12]:
n_samples = t_u.shape[0]
n_val = int(0.2 * n_samples)
shuffled_indices = torch.randperm(n_samples)
train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]
train_indices, val_indices

(tensor([ 4,  3,  7,  5, 10,  9,  0,  2,  8]), tensor([1, 6]))

In [13]:
train_t_un = t_un[train_indices]
val_t_un = t_un[val_indices]
train_t_c = t_c[train_indices]
val_t_c = t_c[val_indices]

def training_loop(epochs, optimizer, params, train_t_u, train_t_c, val_t_u, val_t_c):
    for epoch in range(1, epochs + 1):
        t_p = model(train_t_u, *params)
        loss = loss_fn(t_p, train_t_c)
        with torch.no_grad():
            validation_loss = loss_fn(model(val_t_u, *params), val_t_c)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if epoch % 500 == 0:
            print("Epoch: {}, Loss: {:.2f}, Validation loss: {:.2f}".format(epoch, loss, 
                                                                            validation_loss))
    return params

params = torch.tensor([1.0, 0.0], requires_grad = True)
params = training_loop(3000, 
                       optim.SGD([params], lr = 1e-2), 
                       params, 
                       train_t_un, train_t_c,
                       val_t_un, val_t_c)
print(params)

Epoch: 500, Loss: 8.27, Validation loss: 1.01
Epoch: 1000, Loss: 3.70, Validation loss: 2.06
Epoch: 1500, Loss: 2.87, Validation loss: 3.63
Epoch: 2000, Loss: 2.72, Validation loss: 4.50
Epoch: 2500, Loss: 2.70, Validation loss: 4.90
Epoch: 3000, Loss: 2.69, Validation loss: 5.08
tensor([  5.5010, -18.3851], requires_grad=True)
